<a href="https://colab.research.google.com/github/YujiSue/BioInfoTools/blob/master/NGS%E8%A7%A3%E6%9E%90onColab(VC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NGS解析用ノート(Variant Call編)
* このノートは、次世代シーケンス (NGS) のデータ解析（主にVariant Call）をGoogle Colab上で行うためのノートです。  
* 解析用のコンピュータがない、次世代シーケンスのデータ解析がどんなものか試してみたい、といった方が対象になります。  
<font color="red">※Colabで使えるリソースの制限上、大きなサイズのシーケンスデータの解析や、多量のメモリを必要とする解析などは、時間がかかったり途中でエラーが出る可能性があります。</font>  


## はじめに
GoogleColabの詳しい使用法などは、[本家](https://colab.research.google.com/notebooks/welcome.ipynb)を参照してください。  
簡潔に言うと左下に見えているような![](https://drive.google.com/uc?id=1XWYv4yXxFkzyzxvurd04FT6Qld3-v83O)ボタン(実行ボタン)を押すと、プログラムが実行されます。  
無料でそれなりの性能が利用できるので、お試しにはちょうど良いです。   
さすがにヒトゲノムを対象にあれこれ詳細な解析しようとすると厳しいですが、筆者が良く使う線虫くらいのゲノムサイズ(~100Mb)なら、Google Colabだけで完結して論文に使えるデータを出すことも可能です。

In [ ]:
#@title まずは左にある実行ボタンを押して、必要なファイルを読み込んでください
#@markdown * このノートで使用できるソフトとバージョン一覧<br>
#@markdown >[SRAtools](https://github.com/ncbi/sra-tools): v2.11.2<br>
#@markdown [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/): v0.11.9<br>
#@markdown [fastp](https://github.com/OpenGene/fastp): v0.23.2<br>
#@markdown [BWA](https://github.com/lh3/bwa): v0.7.17<br>
#@markdown [TVC (TorrentSuite)](https://github.com/iontorrent/TS): v5.12.1<br>
#@markdown [Samtools](http://www.htslib.org/): v1.14<br>
#@markdown [BCFtools](http://www.htslib.org/): v1.14<br>
#@markdown [Picard](https://github.com/broadinstitute/picard): v2.26.9<br>
#@markdown [GATK](https://gatk.broadinstitute.org/hc/en-us): v4.2.4.0<br>
#@markdown [Bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml): v2.4.4<br>
#@markdown [IVC (Illumina strelka)](https://github.com/Illumina/strelka): v2.9.x<br>
#@markdown [STAR](https://github.com/alexdobin/STAR): v2.7.9a<br>

#!pip install -q git+https://github.com/YujiSue/ysngs.git
#import ysngs
#from ysngs import config
#from ysngs import installer
#from ysngs import apprun
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/common.py
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/install.py
!wget https://raw.githubusercontent.com/YujiSue/ysngs/main/ysngs/run.py

import os
import common
import install
import run
cfg = common.config()
cfg.setWorkSpace('/content')
cfg.setDefault()
cfg.makeDirs()
loader = install.installer(cfg)
loader.install('SAM')
app = run.apprun(cfg)

## 使用方法

### 1. 解析対象のファイルを準備する
以下の３つの方法のうち、どれか１つ選んで解析対象のファイルを用意してください

#### 1-1. Samtoolに付随するサンプルを利用する（チュートリアル）
【2. リファレンスファイルを準備する】をスキップして、【3. マッピングとVCFの出力】まで進んでください。


#### 1-2. NCBI SRAから解析してみたいファイルをダウンロードする


In [ ]:
#@title ダウンロードしたいIDを指定して実行ボタンを押してください
SRA_ID = 'SRR000001' #@param {type:'string'}
loader.install('SRA')
!vdb-config --interactive
res = app.downloadSRA(srid=SRA_ID, output='/content/Sample')

#### 1-3. 自前のデータを利用する


Google DriveやGoogle Cloud Storageにデータがある場合は、該当するファイルをColabのストレージにコピーしてください。  
その他のクラウドストレージや、サーバ上にあるファイルはwgetやcurlコマンドでコピーしてください。  
ローカルマシン上のファイルのアップロードは時間がかかるのでお勧めしません。

### 2. リファレンスファイルを準備する
NCBIのRefSeqなどからゲノム配列を入手してください。  
ヒト、マウス、線虫のゲノムは下セルのプルダウンから選択することで、ダウンロードできます。  

In [3]:
import os
#@title 
生物種 = '\u7DDA\u866B' #@param['ヒト','マウス','線虫']
その他 = '' #@param {type:'string'}
染色体名の接頭辞 = '' #@param {type:'string'}
染色体名の置換 = False #@param {type:'boolean'}
if 生物種 == 'ヒト':
  os.system('wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/GRCh38.primary_assembly.genome.fa.gz -O '+os.path.join(cfg.REFERENCE_DIR, 'GRCh38.p13.fna.gz'))
  os.system('gunzip '+os.path.join(cfg.REFERENCE_DIR, 'GRCh38.p13.fna.gz'))
elif 生物種 == 'マウス':
  os.system('wget https://ftp.ncbi.nlm.nih.gov/genomes/refseq/vertebrate_mammalian/Mus_musculus/latest_assembly_versions/GCF_000001635.27_GRCm39/GCF_000001635.27_GRCm39_genomic.fna.gz -O '+os.path.join(cfg.REFERENCE_DIR, 'GRCm39.fna.gz'))
  os.system('gunzip '+os.path.join(cfg.REFERENCE_DIR, 'GRCm39.fna.gz'))
elif 生物種 == '線虫':
  os.system('wget ftp://ftp.wormbase.org/pub/wormbase/releases/WS283/species/c_elegans/PRJNA13758/c_elegans.PRJNA13758.WS283.genomic.fa.gz -O '+os.path.join(cfg.REFERENCE_DIR, 'WS283.fa.gz'))
  os.system('gunzip '+os.path.join(cfg.REFERENCE_DIR, 'WS283.fa.gz'))
else:
  os.system('wget '+その他)


### 3. マッピングとVCFの出力
いよいよ、実際にバリアントのリスト（VCFファイル）を出力します。  
一般的な次世代シーケンスの生データ、FastQ (.fq/.fastq)ファイルに保存された配列をリファレンスゲノムにマッピングし、BAMと呼ばれるファイルを生成、さらにバリアントを検出して保存します。

#### パイプライン図
流れとしては以下の図のようになります。

<img src="https://github.com/YujiSue/BioInfoTools/blob/master/image/VC%20pipeline.png?raw=true" width="50%">

#### 実行プログラム
以下のエリアで個別にパラメータをセットしていきます。  
上で1-1 (チュートリアル)を選んだ人は、とくに何もせずに下の実行ボタンを押してください。


In [ ]:
#@title パラメータをセットして左の実行ボタンを押してください
pref = {}
pref['root'] = '/content'
#@markdown * マッピングの設定
マッピング = True #@param {type: 'boolean'}
pref['mapping'] = マッピング
マッピングソフト = 'BWA' #@param ['BWA', 'Bowtie2']
pref['mapper'] = マッピングソフト
if マッピングソフト == 'BWA':
  loader.install('BWA')
elif マッピングソフト == 'Bowtie2':
  loader.install('Bowtie2')
ターゲットシーケンス = False #@param {type: 'boolean'}
pref['targeted_seq'] = ターゲットシーケンス
ターゲット領域 = '' #@param {type: 'string'}
pref['target'] = ターゲット領域
リードグループ情報付加 = True #@param {type: 'boolean'}
pref['add_rg'] = リードグループ情報付加
リードグループID = 'test' #@param {type: 'string'}
pref['read_group_id'] = リードグループID
サンプル名 = 'ex1' #@param {type: 'string'}
pref['sample_name'] = サンプル名
プラットフォーム = 'illumina' #@param {type: 'string'}
pref['platform'] = プラットフォーム
#@markdown * リファレンスの情報
リファレンスファイル = 'ex1.fa' #@param {type: 'string'}
リファレンス名 = 'ex1' #@param {type: 'string'}
pref['refdir'] = '/content/Reference',
pref['reference'] = os.path.join('/content/Reference', リファレンスファイル)
pref['refname'] = リファレンス名
#@markdown * バリアント検出
バリアント検出ソフト = 'GATK' #@param ['GATK', 'BCF', 'TVC']
if バリアント検出ソフト == 'GATK':
  loader.install('GATK')
elif バリアント検出ソフト == 'TVC':
  loader.install('TVC')
elif バリアント検出ソフト == 'BCF':
  loader.install('BCF')
パラメータセット = '' #@param {type: 'string'}
モチーフファイル = '' #@param {type: 'string'}
pref['paramdir'] = '/content/Preference'
pref['vcaller'] = バリアント検出ソフト
pref['vcparam'] = パラメータセット
pref['vcmotif'] = モチーフファイル
#@markdown * シーケンスデータの情報
シーケンスデータの形式 = 'fastq' #@param['fastq', 'bam']
シーケンスデータの種類 = 'single' #@param['single', 'paired']
シーケンスファイル１ = '/content/Test/ex1.fq' #@param {type: 'string'}
シーケンスファイル２ = '' #@param {type: 'string'}
pref['iformat'] = シーケンスデータの形式
pref['seqtype'] = シーケンスデータの種類
if pref['seqtype'] == 'single':
  pref['inputs'] = [シーケンスファイル１]
else:
  pref['inputs'] = [シーケンスファイル１,シーケンスファイル２]   
#@markdown * 結果ファイルの情報
出力ファイル名 = 'test' #@param {type: 'string'}
pref['outdir'] = '/content/Output'
pref['output_name'] = 出力ファイル名
#@markdown オプション
アダプター除去 = False #@param {type: 'boolean'}
pref['cutadapt'] = アダプター除去
pref['adapter_site'] = 'both'
アダプター配列 = ''#@param {type: 'string'}
pref['adapter_seq'] = アダプター配列
クオリティチェック = True #@param {type: 'boolean'}
pref['fqvisualize'] = クオリティチェック
if pref['fqvisualize']:
  loader.install('FQC')
クオリティフィルタ = True #@param {type: 'boolean'}
pref['fqfliter'] = クオリティフィルタ
if pref['fqfliter']:
  loader.install('FP')
シーケンス長下限値 = 20 #@param {type: 'raw'}
pref['fq_minlen'] = シーケンス長下限値
シーケンスクオリティ下限値 = 15 #@param {type: 'raw'}
pref['fq_qual'] = シーケンスクオリティ下限値
シーケンスの単調性 = 30 #@param {type: 'raw'}    
pref['fq_complex'] = シーケンスの単調性
重複リード検出 = True #@param {type: 'boolean'}
pref['mark_dup'] = 重複リード検出
if pref['mark_dup']:
  loader.install('Picard')
スプリットリード検出 = True #@param {type: 'boolean'}
pref['detect_sr'] = スプリットリード検出
シーケンスクオリティの再計算 = False #@param {type: 'boolean'}
pref['recal_seq'] = シーケンスクオリティの再計算
if pref['recal_seq']:
  loader.install('GATK')
pref['use_hotspot'] = False
既知の変異 = '' #@param {type: 'string'}
pref['known_site'] = 既知の変異
バリアントクオリティの再計算 = False #@param {type: 'boolean'}
pref['recal_var'] = バリアントクオリティの再計算
if pref['recal_var']:
  loader.install('Picard')
  loader.install('GATK')
  pref['resources'] = [
                      'hapmap,known=false,training=true,truth=true,prior=15.0:hapmap_3.3.hg38.sites.vcf.gz',
                      'omni,known=false,training=true,truth=false,prior=12.0:1000G_omni2.5.hg38.sites.vcf.gz',
                      '1000G,known=false,training=true,truth=false,prior=10.0:1000G_phase1.snps.high_confidence.hg38.vcf.gz',
                      'dbsnp,known=true,training=false,truth=false,prior=2.0:Homo_sapiens_assembly38.dbsnp138.vcf'
                      ]
  os.system('gsutil cp gs://genomics-public-data/resources/broad/hg38/v0/hapmap_3.3.hg38.vcf.gz /content/Reference')
  os.system('gsutil cp gs://genomics-public-data/resources/broad/hg38/v0/1000G_omni2.5.hg38.vcf.gz /content/Reference')
  os.system('gsutil cp gs://genomics-public-data/resources/broad/hg38/v0/1000G_phase1.snps.high_confidence.hg38.vcf.gz /content/Reference')
  os.system('gsutil cp gs://genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf /content/Reference')
pref['has_control'] = False
pref['control_input'] = ''
スレッド数 =   16#@param {type: 'raw'}
pref['thread_num'] = スレッド数
使用RAM =  8#@param {type: 'raw'}
pref['use_ram'] = 使用RAM
pref['gpgpu'] = False  
#コマンドのみ出力 = False #@param {type: 'boolean'}
pref['cmd_only'] = False
pref['verbose'] = True
pref['tmp'] = '',
pref['mediates'] = []
pref['run_my_script'] = False
途中産物のファイルを削除 = True #@param {type: 'boolean'}
pref['remove_mediates'] = 途中産物のファイルを削除

%cd /content
from common import config
from run import apprun
import getopt
import os
import sys

def makeReadGroupInfo(pref):
  return { 
    'ID': pref['read_group_id'], 
    'SM': pref['sample_name'],
    'PL': pref['platform']
  }

def preprocess(pref, app):
  # Visualize QC
  if pref['fqvisualize']:
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      odir = os.path.join(name + '_QC')
      os.makedirs(odir, exist_ok=True)
      app.runFastQC(input = f, output = odir)
  # Cut adapter
  if pref['cutadapt'] and pref['iformat'] == 'fastq':
    newinputs = []
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      ofile = name + '_cut' + ext
      app.runCutter(adaptor = pref['adapter_seq'], site = pref['adapter_site'], input = f, output = ofile)
      pref['mediates'].append(ofile)
      newinputs.append(ofile)
    pref['inputs'] = newinputs
  # Quality filter
  if pref['fqfliter'] and pref['iformat'] == 'fastq':
    newinputs = []
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      ofile = name + '_filtered' + ext
      app.runFastQFilter(input = f, output = ofile, param = {
        'min_qual': pref['fq_qual'], 'min_len': pref['fq_minlen'], 'min_complex': pref['fq_complex'], 'thread': pref['thread_num']
      })
      newinputs.append(ofile)
    pref['inputs'] = newinputs
  # Visualize QC
  if pref['fqvisualize']:
    for f in pref['inputs']:
      name, ext = os.path.splitext(f)
      odir = os.path.join(name + '_QC')
      os.makedirs(odir, exist_ok=True)
      app.runFastQC(input = f, output = odir)

def mapping(pref, app):
  pref['rg_info'] = makeReadGroupInfo(pref)
  
  # BAM => Fastq
  if pref['iformat'] == 'bam':
    app.runSamtool2Fq(seqtype = pref['seqtype'], input = pref['inputs'][0], outdir = app.cfg.OUT_DIR, outname = pref['output_name'] + '_raw')
    if pref['seqtype'] == 'single':
      pref['inputs'][0] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw.fq')
    elif pref['seqtype'] == 'paired':
      pref['inputs'][0]= os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw_1.fq')
      pref['inputs'].append(os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw_2.fq'))
    for f in pref['inputs']:
      pref['mediates'].append(f)
  
  # Fastq => SAM
  if len(pref['refname']) == 0:
      pref['refname'], ext = os.path.splitext(os.path.basename(pref['reference']))
  
  # BWA-MEM
  if pref['mapper'] == 'BWA':
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_mapped.sam')
    app.runBWA(seqtype = pref['seqtype'], input = pref['inputs'], ref = pref['refname'], output = pref['output'], \
             option={'thread':pref['thread_num'], 'checksr':pref['detect_sr'], 'refpath':pref['reference'], 'addRG':pref['add_rg'], 'rgroup':pref['rg_info']})
    pref['mediates'].append(pref['output'])
  # Bowtie2
  elif pref['mapper'] == 'Bowtie2':
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_mapped.sam')
    app.runBowtie2(seqtype = pref['seqtype'], input = pref['inputs'], ref = pref['refname'], output = pref['output'], \
             option={'thread':pref['thread_num'], 'checksr':pref['detect_sr'], 'refpath':pref['reference'], 'addRG':pref['add_rg'], 'rgroup':pref['rg_info']})
    pref['mediates'].append(pref['output'])
  # STAR
  elif pref['mapper'] == 'STAR':
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'])
    app.runSTAR(seqtype = pref['seqtype'], input = pref['inputs'], ref = pref['refname'], output = pref['output'], \
             option={'thread':pref['thread_num'], 'refpath':pref['reference'], 'annotate':pref['annotate'], 'annotation':pref['annotation']})
    pref['output'] += '.bam'
  pref['input'] = pref['output']
  os.chdir(app.cfg.WORK_SPACE)

  # SAM -> BAM
  if pref['mapper'] == 'STAR':
    if pref['verbose']:
      print('STAR export sorted BAM.')
  else:  
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_mapped.bam')
    app.runSamtool2BAM(input = pref['input'], output = pref['output'], option={ 'thread':pref['thread_num'] })
    pref['mediates'].append(pref['output'])

    # Sort BAM
    pref['input'] = pref['output']
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_sorted.bam')
    app.runSamtoolSort(input = pref['input'], output = pref['output'], option={
      'thread':pref['thread_num'], 'ram': pref['use_ram']/4
    })
    pref['mediates'].append(pref['output'])
    pref['input'] = pref['output']

  # Mark duplicate
  if pref['mark_dup']:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_marked.bam')
    pref['metric'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_metric.txt')
    app.runPicardMD(input = pref['input'], output = pref['output'], metric = pref['metric'])
    pref['mediates'].append(pref['output'])
    pref['input'] = pref['output']

  # Recalibration sequence quality
  if pref['recal_seq']:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '.bam')
    app.runGATKBRecal(input = pref['input'], output = pref['output'], ref = pref['reference'], known = pref['known_site'], option = {'ram': pref['use_ram'] })
  else:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '.bam')
    os.system('mv ' + pref['input'] + ' ' + pref['output'])
  pref['input'] = pref['output']

  # Make index
  app.runSamtoolIndex(input = pref['input'])
def cleanup(pref):
  for f in pref['mediates']:
    os.system('rm ' + f)

def varcall(pref, app):
  # Preprocessing of fastq files
  preprocess(pref, app)
  
  # Mapping
  if pref['mapping']:
    mapping(pref, app)
  else:
    pref['input'] = pref['inputs'][0]
  
  # Variant call
  if pref['recal_var']:
    pref['output']  = os.path.join(app.cfg.OUT_DIR, pref['output_name'] + '_raw')
  else:
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'])
  # GATK HaplotypeCaller
  if pref['vcaller'] == 'GATK':
    gatkopt = {
      #'thread': pref['thread_num'],
      'ram': pref['use_ram']
    }
    if pref['targeted_seq']:
      gatkopt['target'] = pref['target']
    app.runGATKVarCall(input = pref['input'], output = pref['output'], ref = pref['reference'], option = gatkopt)
    pref['input'] = pref['output'] + '.vcf.gz'
  # BCFtools mpileup and call
  elif pref['vcaller'] == 'BCF':
    app.runBCFVarCall(input = pref['input'], output = pref['output'], ref = pref['reference'])
    pref['input'] = pref['output'] + '.vcf.gz'
  # TorrentSuite variant caller
  elif pref['vcaller'] == 'TVC':
    tvcopt = {
      'thread': pref['thread_num'],
      'param' : os.path.join(app.cfg.PREFERENCE_DIR, pref['vcparam']), 
      'motif' : os.path.join(app.cfg.PREFERENCE_DIR, pref['vcmotif']) 
    }
    if pref['targeted_seq']:
      tvcopt['target'] = pref['target']
    if pref['use_hotspot']:
      tvcopt['hotspot'] = pref['known_site']
    if pref['has_control']:
      tvcopt['control'] = pref['control_input']
    app.runTVC(input = pref['input'], output = app.cfg.OUT_DIR, ref = pref['reference'], option = tvcopt)
  # google deep variant
  elif pref['vcaller'] == 'GDV':
    gdvopt = {
      'gpu': pref['gpgpu'],
      'processor': pref['thread_num']
      }
    if pref['targeted_seq']:
      gdvopt['target'] = pref['target']
    app.runGDVCall(input = pref['input'], output = pref['output'], ref = pref['reference'], option = gdvopt)
  
  # Recalibration of variant qualities
  if pref['recal_var']:
    pref['mediates'].append(pref['output'] + '.vcf.gz')
    pref['mediates'].append(pref['output'] + '.vcf.gz.tbi')
    pref['output'] = os.path.join(app.cfg.OUT_DIR, pref['output_name'])
    app.runGATKVRecal(input = pref['input'], output = pref['output'], ref = pref['reference'], resources = pref['resources'], option = {'ram': pref['use_ram']})
    pref['mediates'].append(pref['output'] + '_snp.vcf')
    pref['mediates'].append(pref['output'] + '_indel.vcf')
    os.system('mv ' + pref['output'] + '_recal.vcf ' + pref['output'] + '.vcf')
  
  # Run an additional script
  if pref['run_my_script']:
    os.system('python ' + os.path.join(app.cfg.SCRIPT_DIR, pref['myscript']) + ' ' + pref['myargs'])

  # Erase intermediate files
  if pref['remove_mediates']:
    cleanup(pref)

varcall(pref, app)

In [ ]:
#@title　チュートリアルとして進めてきた人は、左の実行ボタンを押すと出力されたバリアントが確認できます
!gunzip /content/Output/test.vcf.gz
!cat /content/Output/test.vcf

### 4. それ以降の処理

このあと、さらに出力されたVCFに対して  
> - コントロール群（バックグラウンド系統）との差分をとる
> - 既知のSNPとの連鎖解析に基づいた絞り込みをかける
> - 遺伝子アノテーションをする
> - タンパク質機能へ与える影響の重篤度を判定する
> - 解析結果のまとめ図を生成・出力する  
  
などなどの処理を行っていくことになります。  
この辺りは、個々に使用するデータやソフト、スクリプトがさらに細分化されていくので、このノートでは紹介していません。  


## さいごに
今や、お金を出して高性能のコンピュータを購入したり、複雑なプログラムを書かなくても、その気になれば誰でもボタン１つ（+パラメータ設定を少し）で、次世代シーケンスの生データからバリアントの一覧(VCFファイル)を出力できる時代です。  
もちろん実際に論文化しようと思ったら、もう少し後処理が必要なわけですが、それを考慮してもゲノム解析の敷居は急速に低くなっています。  
まだまだこうした解析に躊躇する方もいるとは思いますが、本ノートブックのような手軽な解析プラットフォームが広まることで、生物学・医学とインフォマティクスの協調による成果が増えていくことを願っています。

# 参考にしたサイト  
> - http://bio-bwa.sourceforge.net/bwa.shtml
> - http://www.htslib.org/doc/samtools.html
> - http://www.htslib.org/doc/bcftools.html
> - https://gatk.broadinstitute.org/hc/en-us/articles/360037225632-HaplotypeCaller
> - http://bowtie-bio.sourceforge.net/bowtie2/manual.shtml
> - https://github.com/broadinstitute/picard
> - https://assets.thermofisher.com/TFS-Assets/LSG/manuals/MAN0017972_031419_TorrentSuite_5_12_UG_.pdf



# このノートについて
* [<b><u>Suehiro Y</u></b>, Yoshina S, Motohashi T, Iwata S, Dejima K, Mitani S. Efficient collection of a large number of mutations by mutagenesis of DNA damage response defective animals. Sci Rep. 2021 Apr 7;11(1):7630.](https://pubmed.ncbi.nlm.nih.gov/33828169/)  

* 現在作成中の別の論文中で、このノートの英語版かつ完全版のリンクを公開予定
